In [7]:
import numpy as np
import integral_image_transform
import sklearn.datasets

def get_faces():
    return sklearn.datasets.fetch_olivetti_faces()

images = get_faces()['images']
labels = get_faces()['target']

In [10]:
def integral_transform(img, square = True, ravelled = False, dims = None):
    if square and ravelled:
        side_len = int(len(img) ** .5)
        img_reshaped = np.reshape(img, (side_len, side_len))
        ans = np.zeros((side_len, side_len))
        for i in range(2*side_len - 1):
            for j in range(i + 1):
                if j < side_len and i - j < side_len:
                    if i == 0:
                        ans[0][0] = img_reshaped[0][0]
                    elif j == 0:
                        ans[j][i-j] = ans[j][i-j-1] + img_reshaped[j][i-j]
                    elif i == j:
                        ans[j][i-j] = ans[j-1][i-j] + img_reshaped[j][i-j]
                    else:
                        ans[j][i-j] = ans[j][i-j-1] + ans[j-1][i-j] - ans[j-1][i-j-1] + img_reshaped[j][i-j]
    if square and not ravelled:
        side_len = int(len(img))
        ans = np.zeros((side_len, side_len))
        for i in range(2*side_len - 1):
            for j in range(i + 1):
                if j < side_len and i - j < side_len:
                    if i == 0:
                        ans[0][0] = img[0][0]
                    elif j == 0:
                        ans[j][i-j] = ans[j][i-j-1] + img[j][i-j]
                    elif i == j:
                        ans[j][i-j] = ans[j-1][i-j] + img[j][i-j]
                    else:
                        ans[j][i-j] = ans[j][i-j-1] + ans[j-1][i-j] - ans[j-1][i-j-1] + img[j][i-j]

    return ans




In [12]:
print integral_transform(images[0]).shape

(64, 64)


In [ ]:
def compute_two_rectangle_features(image):
    # Calculating at every point and scale for now
    length,width = image.shape
    
    


compute_two_rectangle_features(images[0])
    

In [8]:
print images[0].shape

(64, 64)
